In [ ]:
import zipfile
import pandas as pd
from PIL import Image
import requests
import tensorflow as tf
from transformers import CLIPProcessor, CLIPModel, BertTokenizer, TFBertModel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [ ]:
from google.colab import drive
import shutil

# Подключение к Google Диску
drive.mount('/content/drive')

# Путь к файлу на Google Диске
file_path = "https://drive.google.com/file/d/1c_I-yjBXaGbGgiiqphSGpq1ja_OMG2CD/view?usp=sharing"  # Замените этот путь на фактический путь вашего файла на Google Диске


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import pickle

with open('/content/drive/MyDrive/train.tsv', 'rb') as f:
    bert_features_list = pickle.load(f)
with open('/content/drive/MyDrive/test_no_target.tsv', 'rb') as f:
    clip_features_list = pickle.load(f)

In [ ]:
# Загрузка модели CLIP
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Загрузка модели BERT
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

**не запускать**

генерация призаков для трейна

In [ ]:
# Initialize lists to store features and labels
X_data = []
y_data = []
# Открываем файл train.tsv и загружаем его в DataFrame
df = pd.read_csv("train.tsv", delimiter="\t")
# Открываем архив zip
filepath = '/content/drive/MyDrive/train.zip'
processed_images = 0  # Счетчик обработанных изображений
with zipfile.ZipFile(filepath, "r") as zip_ref:
    total_images = len(zip_ref.namelist())
    # Проходим по всем файлам в архиве
    for filename in zip_ref.namelist():
        try:
            # Извлекаем изображение из архива
            with zip_ref.open(filename) as img_file:
                # Проверяем, что файл - PNG изображение
                if filename.endswith(".png"):
                    # Увеличиваем счетчик обработанных изображений
                    processed_images += 1

                    # Открываем изображение
                    image = Image.open(img_file)

                    # Находим соответствующий текст из train.tsv
                    text = df.loc[df['filename'] == filename, 'text'].iloc[0]

                    # Создаем входные данные для модели CLIP
                    inputs_clip = processor(text=[text], images=image, return_tensors="pt")

                    # Получаем признаки изображения с помощью модели CLIP
                    image_features_clip = clip_model.get_image_features(inputs_clip.pixel_values)

                    # Получаем признаки текста с помощью модели BERT
                    tokenized_texts = bert_tokenizer(text, return_tensors="tf", padding=True, truncation=True)
                    text_features_bert = bert_model(**tokenized_texts).pooler_output

                    # Объединяем признаки изображения и текста
                    combined_features = tf.concat([image_features_clip.detach().numpy(), text_features_bert.numpy()], axis=1)

                    # Получаем метку для изображения из train.tsv
                    mark = df.loc[df['filename'] == filename, 'mark'].iloc[0]

                    # Добавляем признаки и метку в список
                    X_data.append(combined_features)
                    y_data.append(mark)

                    # print(f"\rProcessed {processed_images}/{total_images} images", end='', flush=True)

                    # # Выводим информацию о фотографии
                    # print("\nFilename:", filename)
                    # print("Text:", text)
                    # print("Mark:", mark)
                    # print("Combined Features:", combined_features)


                    # Выводим текущий прогресс, перезаписывая строку
                    print(f"\rProcessed {processed_images}/{total_images} images", end='', flush=True)
        except Exception as e:
            print("\rError processing image:", e)
            continue  # Пропустить обработку текущего изображения и перейти к следующему


Processed 72/25973 images

Token indices sequence length is longer than the specified maximum sequence length for this model (85 > 77). Running this sequence through the model will result in indexing errors


Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: unrecognized data stream contents when reading image file
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: Bad CRC-32 for file 'train/competition_data:322856.png'
Error processing image: mean must have 1 elements if it is

In [ ]:
import pickle
# Сохраняем данные в файлы на Google Диск
with open('/content/drive/MyDrive/X_data.pkl', 'wb') as f:
    pickle.dump(X_data, f)

with open('/content/drive/MyDrive/y_data.pkl', 'wb') as f:
    pickle.dump(y_data, f)

In [ ]:
import pickle

with open('/content/drive/MyDrive/X_data.pkl', 'rb') as f:
    X_data = pickle.load(f)
with open('/content/drive/MyDrive/y_data.pkl', 'rb') as f:
    y_data = pickle.load(f)

In [ ]:
import numpy as np
X_data1 = np.vstack(X_data)
y_data1=np.vstack(y_data)
# Теперь X_data является двумерным массивом, где каждая строка - это объект
print(X_data1.shape)

(25911, 1280)


In [ ]:
from sklearn.decomposition import PCA
import numpy as np

# Предположим, X_data - ваш массив данных (12000 x 1280)
X_data = X_data1  # Заглушка для демонстрации

# Инициализация PCA для сокращения до 350 признаков
pca = PCA(n_components=492)

# Применение PCA к данным
X_reduced = pca.fit_transform(X_data)

# Проверка результата
print("Форма исходных данных:", X_data.shape)
print("Форма данных после PCA:", X_reduced.shape)


Форма исходных данных: (25911, 1280)
Форма данных после PCA: (25911, 492)


отбор признаков для трейна (не понадобился)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

# Предположим, X_data1 - ваш массив из массивов (25,911 x 1280)
# y_data1 - массив целевых переменных для каждого подмассива, размером 25,911

selected_features = np.zeros((25911, 350))  # Итоговый массив для хранения отобранных признаков

for i in range(len(X_data1)):
    # Обучение случайного леса для каждого подмассива
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_data1[i].reshape(1, -1), y_data1[i])  # Применяем reshape, если каждый подмассив представляет один объект

    # Отбор 350 наиболее важных признаков
    model = SelectFromModel(rf, max_features=350, prefit=True)
    important_indices = model.get_support(indices=True)

    # Сохранение отобранных признаков
    selected_features[i, :] = X_data1[i, important_indices]

# Теперь selected_features содержит только отобранные признаки для каждого объекта


In [ ]:
import pickle
# Сохраняем данные в файлы на Google Диск
with open('/content/drive/MyDrive/selected_features1.pkl', 'wb') as f:
    pickle.dump(selected_features, f)


In [ ]:
print(selected_features.shape)

(25911, 350)


данные для обучения

In [ ]:
import pickle
with open('/content/drive/MyDrive/X_selected.pkl', 'rb') as f:
    X_selected = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/X_test_data.pkl', 'rb') as f:
    X_data = pickle.load(f)

In [ ]:
X_data = np.array(X_data)
X_data = np.squeeze(X_data, axis=1)
X_data.shape

(12000, 1280)

In [ ]:
import numpy as np
from scipy.stats import pearsonr

# Предполагается, что X_data - это двумерный массив (n_samples, n_features) признаков

# Матрица для хранения индексов отобранных признаков для каждого наблюдения
selected_features_indices = np.zeros((X_data.shape[1], 500), dtype=int)

# Итерация по каждому признаку
for i in range(X_data.shape[1]):
    correlations = []

    # Вычисление корреляции каждого признака с другими признаками
    for j in range(X_data.shape[1]):  # Перебор признаков
        if i != j:  # Исключаем корреляцию признака с самим собой
            correlation = pearsonr(X_data[:, i], X_data[:, j])[0]
            correlations.append((j, correlation))

    # Сортировка признаков по абсолютному значению корреляции
    sorted_features = sorted(correlations, key=lambda x: abs(x[1]), reverse=True)

    # Выбор индексов 300 наиболее коррелирующих признаков
    selected_features_indices[i, :] = [index for index, _ in sorted_features[:500]]

# Теперь selected_features_indices - это двумерный массив, где каждая строка содержит индексы
# 300 наиболее коррелирующих признаков для каждого признака

# Создаем новый массив, чтобы разместить отобранные признаки
selected_features_data2 = np.zeros((X_data.shape[0], 500))

# Итерация по каждому сэмплу
for i in range(X_data.shape[0]):
    # Получаем индексы 300 наиболее коррелирующих признаков для данного сэмпла
    indices = selected_features_indices[i, :]

    # Извлекаем только соответствующие отобранные признаки из X_data и размещаем их в новом массиве
    selected_features_data2[i] = X_data[i, indices]

# selected_features_data теперь содержит только 300 наиболее коррелирующих признаков для каждого сэмпла


In [ ]:
import pickle
# Сохраняем данные в файлы на Google Диск
with open('/content/drive/MyDrive/selected_features_test3.pkl', 'wb') as f:
    pickle.dump(selected_features_data2, f)

обучение логрега

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X_reduced, y_data, test_size=0.2, random_state=42)

# Инициализация модели логистической регрессии с L2 регуляризацией и C=1e-4
model = LogisticRegression(penalty='l2', C=1, multi_class='multinomial', max_iter=100, random_state=42)

# Обучение модели
model.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(X_test)

# Оценка производительности модели
accuracy = accuracy_score(y_test, y_pred)
print("Точность модели на тестовых данных:", accuracy)


Точность модели на тестовых данных: 0.5772718502797608


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
logreg_pred= model.predict(X_reduced)

In [ ]:
print(len(X_data),
len(y_data))

25911 25911


In [ ]:
import numpy as np

# Получение уникальных меток и их количество
unique_labels, counts = np.unique(df, return_counts=True)

# Вывод уникальных меток и их количество
for label, count in zip(unique_labels, counts):
    print(f'Метка: {label}, Количество: {count}')


Выходные данные были обрезаны до нескольких последних строк (5000).
Метка: test/competition_data:291151.png	Удовлетворительно, Количество: 1
Метка: test/competition_data:291171.png	Удовлетворительно, Количество: 1
Метка: test/competition_data:291178.png	Удовлетворительно, Количество: 1
Метка: test/competition_data:291199.png	Идеально, Количество: 1
Метка: test/competition_data:291242.png	Плохо, Количество: 1
Метка: test/competition_data:29127.png	Удовлетворительно, Количество: 1
Метка: test/competition_data:291285.png	Идеально, Количество: 1
Метка: test/competition_data:291345.png	Удовлетворительно, Количество: 1
Метка: test/competition_data:291355.png	Удовлетворительно, Количество: 1
Метка: test/competition_data:291356.png	Удовлетворительно, Количество: 1
Метка: test/competition_data:291366.png	Удовлетворительно, Количество: 1
Метка: test/competition_data:291384.png	Удовлетворительно, Количество: 1
Метка: test/competition_data:291402.png	Удовлетворительно, Количество: 1
Метка: test/co

# модели

**Классификатор**

In [ ]:
from tensorflow.keras.utils import to_categorical

# Предположим, y_train и y_val содержат целочисленные метки от 0 до 2
y_train_one_hot = to_categorical(y_train, num_classes=3)
y_val_one_hot = to_categorical(y_test, num_classes=3)

классификатор нейросетевой

In [ ]:
from tensorflow.keras import regularizers
num_classes = 3
# Определение классификатора
dropout_rate = 0.5  # Процент отключаемых нейронов, обычно 0.2 - 0.5
l2_lambda = 5e-5  # Сила регуляризации L2, экспериментируйте с этим значением

# Определение модели
classifier = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=((None,500))),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Компиляция классификатора
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import ProgbarLogger, Callback

# Определение кастомного обратного вызова для контроля прекращения обучения
class MyCustomCallback(Callback):
    def __init__(self, target_accuracy=0.95, max_epochs=20):
        super(MyCustomCallback, self).__init__()
        self.target_accuracy = target_accuracy
        self.max_epochs = max_epochs
        self.stop_training = False

    def on_epoch_end(self, epoch, logs=None):
        if logs.get('val_accuracy') >= self.target_accuracy:
            print(f"\nReached target accuracy of {self.target_accuracy}. Stopping training.")
            self.model.stop_training = True
        elif epoch >= self.max_epochs - 1:
            print(f"\nReached maximum epochs ({self.max_epochs}). Stopping training.")
            self.model.stop_training = True

# Количество эпох
epochs = 8

# Создание списка обратных вызовов с двумя кастомными обратными вызовами
callbacks = [
    MyCustomCallback(),  # Ваш кастомный обратный вызов
    ProgbarLogger()  # Обратный вызов для отображения прогресса обучения
]

# Обучение модели с использованием списка обратных вызовов
history = classifier.fit(X_train, y_train_one_hot, epochs=epochs, batch_size=432, validation_data=(X_test, y_val_one_hot), callbacks=callbacks)

Epoch 1/8
48/48 [==============================] - 4s 75ms/sample - loss: 149.1248 - accuracy: 0.4282 - val_loss: 10.1296 - val_accuracy: 0.4561
Epoch 2/8
48/48 [==============================] - 2s 36ms/sample - loss: 29.1022 - accuracy: 0.4260 - val_loss: 21.0585 - val_accuracy: 0.4644
Epoch 3/8
48/48 [==============================] - 2s 37ms/sample - loss: 17.4477 - accuracy: 0.4252 - val_loss: 32.9483 - val_accuracy: 0.4644
Epoch 4/8
48/48 [==============================] - 3s 54ms/sample - loss: 10.5993 - accuracy: 0.4350 - val_loss: 5.4801 - val_accuracy: 0.4561
Epoch 5/8
48/48 [==============================] - 2s 46ms/sample - loss: 7.3512 - accuracy: 0.4274 - val_loss: 2.3230 - val_accuracy: 0.4644
Epoch 6/8
48/48 [==============================] - 2s 39ms/sample - loss: 1.6327 - accuracy: 0.4396 - val_loss: 1.0741 - val_accuracy: 0.4644
Epoch 7/8
48/48 [==============================] - 2s 38ms/sample - loss: 0.9749 - accuracy: 0.4639 - val_loss: 0.9344 - val_accuracy: 0.464

In [ ]:
import pickle
with open('/content/drive/MyDrive/selected_features_data1.pkl', 'rb') as f:
    selected_features_data = pickle.load(f)
with open('/content/drive/MyDrive/y_data2.pkl', 'rb') as f:
    y_data = pickle.load(f)

In [ ]:
y_data.shape

In [ ]:

# Разделение данных на обучающую и валидационную выборк
X_train, X_test, y_train, y_test = train_test_split(selected_features_data, y_data, test_size=0.2, random_state=42)

In [ ]:
X_train = np.squeeze(X_train, axis=1)
X_val = np.squeeze(X_val, axis=1)

In [ ]:
y_train.shape

(20728,)

In [ ]:
y_data.shape

(25911,)

In [ ]:
X_train.shape

(20728, 1280)

**не запускать**

признаки для теста

In [ ]:
import torch
import zipfile
import pandas as pd
from PIL import Image
from transformers import AutoModel, AutoTokenizer, CLIPModel, CLIPProcessor
from sklearn.decomposition import PCA

# Load models
bert_model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")
bert_tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Read data
df = pd.read_csv("test_no_target.tsv", delimiter="\t")
filepath = '/content/drive/MyDrive/test.zip'
X_test_data = []

with zipfile.ZipFile(filepath, 'r') as zip_ref:
    total_images = len([name for name in zip_ref.namelist() if name.endswith('.png')])
    processed_images = 0

    for filename in zip_ref.namelist():
      try:
          if filename.endswith('.png'):
              with zip_ref.open(filename) as img_file:
                  image = Image.open(img_file).convert("RGB")

                  # Find the corresponding text from train.tsv
                  text = df.loc[df['filename'] == filename, 'text'].iloc[0]

                  # Process image and text using CLIP
                  inputs_clip = clip_processor(text=[text], images=[image], return_tensors="pt")
                  image_features_clip = clip_model.get_image_features(inputs_clip.pixel_values)

                  # Process text using BERT
                  bert_inputs = bert_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
                  text_features_bert = bert_model(**bert_inputs).last_hidden_state.mean(dim=1)

                  # Concatenate features
                  combined_features = torch.cat((text_features_bert, image_features_clip), dim=1).detach().numpy()

                  # Save features and labels
                  X_test_data.append(combined_features)

                  processed_images += 1
                  print(f"\rProcessed {processed_images}/{total_images} images. Combined features size: {combined_features.shape}", end='', flush=True)
      except Exception as e:
            print("\rError processing image:", e)
            X_test_data.append(np.zeros(combined_features.shape))
print("\nProcessing complete.")


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Processed 51/12000 images. Combined features size: (1, 1280)

Token indices sequence length is longer than the specified maximum sequence length for this model (84 > 77). Running this sequence through the model will result in indexing errors


Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an iterable, got 3
Error processing image: mean must have 1 elements if it is an it

In [ ]:
import pickle
# Сохраняем данные в файлы на Google Диск
with open('/content/drive/MyDrive/X_test_data2.pkl', 'wb') as f:
    pickle.dump(X_test_data, f)

In [ ]:
import pickle
with open('/content/drive/MyDrive/test_have_target9.tsv', 'rb') as f:
  test_have_target9 = pickle.load(f)

In [ ]:
import pandas as pd
import random

# Создаем DataFrame
df = pd.read_csv('test_no_target.tsv', sep='\t')

# Создаем список слов "Удовлетворительно"
udovletvoritelno_count = int(len(df) * 0.6)
udovletvoritelno = ["Удовлетворительно"] * udovletvoritelno_count

# Создаем список слов "Идеально"
idealno_count = int(len(df) * 0.3)
idealno = ["Идеально"] * idealno_count

# Создаем список слов "Плохо"
ploho_count = int(len(df) * 0.1)
ploho = ["Плохо"] * ploho_count

# Соединяем все списки вместе и перемешиваем
marks = udovletvoritelno + idealno + ploho
random.shuffle(marks)

# Добавляем столбец "mark" в DataFrame
df['mark'] = marks




# Удаление колонки 'text'
df.drop('text', axis=1, inplace=True)

# Сохранение измененных данных обратно в файл CSV до применения операции replace
df.to_csv('test_target15.tsv', sep='\t', index=False)


In [ ]:
df.head()

,filename,mark
0,test/competition_data:117096.png,Удовлетворительно
1,test/competition_data:196699.png,Удовлетворительно
2,test/competition_data:242127.png,Удовлетворительно
3,test/competition_data:292007.png,Идеально
4,test/competition_data:51895.png,Удовлетворительно


In [ ]:
df = pd.read_csv('test_have_target11.tsv')
df.drop('mark', axis=1, inplace=True)
#df.to_csv('test_have_target11.tsv', index=False)

In [ ]:
df.head()

,filename\tmark
0,test/competition_data:117096.png\tИдеально
1,test/competition_data:196699.png\tУдовлетворит...
2,test/competition_data:242127.png\tУдовлетворит...
3,test/competition_data:292007.png\tУдовлетворит...
4,test/competition_data:51895.png\tПлохо


/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 9 (	) missing from current font.
  plt.savefig(


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('filename\tmark').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

# предикты

In [ ]:
import pickle
with open('/content/drive/MyDrive/X_test_data2.pkl', 'rb') as f:
  X_test_data = pickle.load(f)

In [ ]:
X_test_data = np.array(X_test_data)

In [ ]:
X_test_data = np.squeeze(X_test_data, axis=1)

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

# Предположим, X_data - ваш массив данных (12000 x 1280)
X_data = X_test_data  # Заглушка для демонстрации

# Инициализация PCA для сокращения до 350 признаков
pca = PCA(n_components=492)

# Применение PCA к данным
X_reduced = pca.fit_transform(X_data)

# Проверка результата
print("Форма исходных данных:", X_data.shape)
print("Форма данных после PCA:", X_reduced.shape)


Форма исходных данных: (12000, 1280)
Форма данных после PCA: (12000, 492)


In [ ]:
X_reduced

array([[-3.54861698, -0.80145113, -0.3147823 , ..., -0.23329322,
        -0.15787373, -0.08415245],
       [ 7.16505071,  0.87999586, -1.30546352, ...,  0.01000399,
        -0.05507918,  0.01270196],
       [ 0.05304899,  3.52671923, -1.70422604, ..., -0.01452498,
         0.33731735, -0.16121084],
       ...,
       [-0.23989848,  4.08704879,  2.94958939, ...,  0.21537081,
        -0.06061068, -0.25404982],
       [ 4.21583192, -1.18375829,  3.31486532, ...,  0.07405776,
         0.11650312, -0.28346805],
       [ 5.5938644 , -1.39284298, -0.81060332, ...,  0.00948628,
         0.12505205, -0.05562433]])

In [ ]:
X_reduced.shape

(12000, 459)

In [ ]:
import pandas as pd

df = pd.read_csv('test_no_target.tsv', sep='\t')

# Предполагается, что predicted_classes загружены правильно в этот момент

# Добавление новой колонки 'mark' с переименованными метками
df['mark'] = y_test_pred

# Преобразование числовых значений в колонке 'mark' в текстовые метки
df['mark'] = df['mark'].replace({0: 'Плохо', 1: 'Удовлетворительно', 2: 'Идеально'})

# Удаление колонки 'text'
df.drop('text', axis=1, inplace=True)

# Сохранение измененных данных обратно в файл CSV до применения операции replace
df.to_csv('test_knn_target1.tsv', sep='\t', index=False)


In [ ]:
df['mark'] = df['mark'].replace({0: 'Плохо', 1: 'Удовлетворительно', 2: 'Идеально'})

In [ ]:
df.to_csv('/content/drive/MyDrive/test_knn_target1.tsv', sep='\t', index=False)

In [ ]:
predictions = classifier.predict(X_reduced)

375/375 [==============================] - 2s 4ms/step


In [ ]:
print(len(predictions))

12000


In [ ]:
import numpy as np
predicted_classes = np.argmax(predictions, axis=1)

# Проверим первые несколько предсказанных классов
print(predicted_classes)

In [ ]:
unique_labels, counts = np.unique(predicted_classes, return_counts=True)

# Вывод уникальных меток и их количество
for label, count in zip(unique_labels, counts):
    print(f'Метка: {label}, Количество: {count}')


Метка: 0, Количество: 1955
Метка: 1, Количество: 6644
Метка: 2, Количество: 3401


In [ ]:
print(len(predicted_classes))

12000


In [ ]:
import pandas as pd

df = pd.read_csv('test_no_target.tsv', sep='\t')

# Предполагается, что predicted_classes загружены правильно в этот момент

# Добавление новой колонки 'mark' с переименованными метками
df['mark'] = predicted_classes

# Преобразование числовых значений в колонке 'mark' в текстовые метки
df['mark'] = df['mark'].replace({0: 'Плохо', 1: 'Удовлетворительно', 2: 'Идеально'})

# Удаление колонки 'text'
df.drop('text', axis=1, inplace=True)

# Сохранение измененных данных обратно в файл CSV до применения операции replace
df.to_csv('test_have_target9.tsv', sep='\t', index=False)


In [ ]:
df['mark'] = df['mark'].replace({0: 'Плохо', 1: 'Удовлетворительно', 2: 'Идеально'})

In [ ]:
df.head()

,filename,mark
0,test/competition_data:117096.png,Идеально
1,test/competition_data:196699.png,Удовлетворительно
2,test/competition_data:242127.png,Удовлетворительно
3,test/competition_data:292007.png,Удовлетворительно
4,test/competition_data:51895.png,Плохо


In [ ]:
df.to_csv('/content/drive/MyDrive/test_have_target9.tsv', sep='\t', index=False)

ПОПРОБОВАТЬ Использование весов классов

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Классы
classes = np.unique(y_data)

# Вычисление весов для классов
class_weights = compute_class_weight('balanced', classes=classes, y=y_data.flatten())
class_weights_dict = dict(zip(classes, class_weights))

# Обучение модели с учетом весов классов
history = classifier.fit(X_train, y_train_one_hot, epochs=epochs, batch_size=432, validation_data=(X_val, y_val_one_hot), callbacks=callbacks, class_weight=class_weights_dict)


ПОПРОБОВАТЬ МЕТРИКИ

In [ ]:
from tensorflow.keras.metrics import Precision, Recall
from tensorflow_addons.metrics import F1Score

# Компиляция с новыми метриками
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[F1Score(num_classes=num_classes, average='macro'), Precision(), Recall()])
